### Piesi

In [1]:
import networkx as nx
#import osmnx as ox
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os
from math import sin, cos, sqrt, atan2, radians
#from matplotlib_scalebar.scalebar import ScaleBar
from sklearn.metrics.pairwise import haversine_distances
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.optimize import curve_fit

#%matplotlib inline
#ox.config(log_console=True)
#ox.__version__

# Color blind library
plt.style.use('tableau-colorblind10')          
plt.style.library['tableau-colorblind10']
                

def getDistanceFromLatLonInM(lat1,lon1,lat2,lon2):
    """ Function that returns the distance in metres between 2 GPS locations in degrees (latitude and longitude).
    It is based in the Haversine formula (https://en.wikipedia.org/wiki/Haversine_formula) which takes into account the
    Earth's curvature. 
    
    Input:
        - 2 GPS coordinates: (latitude1,longitude1) of the first point and (latitude2,longitude2) of the second point. 
        
    Output:
        - Distance in metres between the two GPS locations.
    """
    
    R = 6371 # Radius of the earth in km
    dLat = radians(lat2-lat1)
    dLon = radians(lon2-lon1)
    rLat1 = radians(lat1)
    rLat2 = radians(lat2)
    a = sin(dLat/2) * sin(dLat/2) + cos(rLat1) * cos(rLat2) * sin(dLon/2) * sin(dLon/2) 
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    d = R * c # Distance in km
    e= d*1000 #distance in m
    
    return e


In [2]:
import datetime
compiled_data = pd.DataFrame()
all_files = glob.glob(os.path.join("*.csv"),root_dir="./processed and interpolated data") 
for file in all_files:   
    df = pd.read_csv("./processed and interpolated data/"+file)  # Read the csv file of each user
    compiled_data = compiled_data.append(df, ignore_index=True)
#compiled_data["timestamp"] = compiled_data.apply(lambda x: datetime.datetime.strptime(x.time,"%Y-%m-%d %H:%M:%S").timestamp())
#datetime.datetime.strptime(compiled_data["time"],"%Y-%m-%d %H:%M:%S").timestamp()
    
    
compiled_data.to_csv("compiled_data.csv", index=False)

In [3]:
import geopy.distance
import datetime
movement = pd.read_csv("routes_multiple.csv")
lamps = pd.read_csv("AghLamps.csv")
movement.sort_values("time", inplace = True)
lampsOn = pd.DataFrame(columns=['startTime','endTime','latitude','longitude'])

In [20]:



for l in range (1,len(lamps)):
    
    lamp = lamps.iloc[l]    
    
    def lam(x):
        return geopy.distance.distance((lamp.Y,lamp.X),(x['latitude'], x['longitude'])).m<20
    sourceDf = movement.loc[movement.apply(lam,axis=1)]
    if len(sourceDf)<1:
        continue
    
    mintime = datetime.datetime.strptime(sourceDf.iloc[0].time, "%Y-%m-%d %H:%M:%S")
    prevtime = mintime
    for i in range (1,len(sourceDf)):
        current = datetime.datetime.strptime(sourceDf.iloc[i].time, "%Y-%m-%d %H:%M:%S")
        if (((current - datetime.timedelta(seconds=10)) > prevtime) or i == len(sourceDf)-1):
            #save period and startover
            lampsOn.loc[len(lampsOn.index)] = [mintime, prevtime + datetime.timedelta(seconds=10), lamp.Y, lamp.X] 
            mintime = current
            prevtime = mintime
        else:
            prevtime = current
            
lampsOn.to_csv("lampOn.csv", index=False) 


In [22]:
lampsOn.to_csv("lampOn.csv", index=False) 

In [18]:

lamp = (41.31587009,2.02229937)

#lam = lambda x: geopy.distance.distance((41.31587009,2.02229937),(x.latitude, x.longtitude))
def lam(x):
    return geopy.distance.distance((41.31587009,2.02229937),(x['latitude'], x['longitude'])).m<20

rslt_df = compiled_data.loc[compiled_data.apply(lam,axis=1)]
rslt_df.to_csv("lamp1.csv", index=False)


        
#compiled_data["lDist"] = compiled_data.apply(lam,axis=1)
# 225082.10, 5059301.28

In [51]:
lampsOn = pd.DataFrame(columns=['startTime','endTime','latitude','longitude'])

sourceDf = rslt_df.sort_values("time")

mintime = datetime.datetime.strptime(sourceDf.iloc[0].time, "%Y-%m-%d %H:%M:%S")
prevtime = mintime
for i in range (1,len(sourceDf)):
    current = datetime.datetime.strptime(sourceDf.iloc[i].time, "%Y-%m-%d %H:%M:%S")
    print(f"{current}    :     {prevtime}\n{((current - datetime.timedelta(seconds=10)) > prevtime)}\n")
    if ((current - datetime.timedelta(seconds=10)) > prevtime):
        #save period and startover
        lampsOn.loc[len(lampsOn.index)] = [mintime, prevtime + datetime.timedelta(seconds=10), lamp[0], lamp[1]] 
        mintime = current
        prevtime = mintime
    else:
        prevtime = current
lampsOn.to_csv("lampOn.csv", index=False)

2018-11-05 18:23:09    :     2018-11-05 18:23:08
False

2018-11-05 18:23:10    :     2018-11-05 18:23:09
False

2018-11-05 18:23:11    :     2018-11-05 18:23:10
False

2018-11-05 18:23:12    :     2018-11-05 18:23:11
False

2018-11-05 18:23:13    :     2018-11-05 18:23:12
False

2018-11-05 18:23:14    :     2018-11-05 18:23:13
False

2018-11-05 18:23:15    :     2018-11-05 18:23:14
False

2018-11-05 18:23:16    :     2018-11-05 18:23:15
False

2018-11-05 18:23:17    :     2018-11-05 18:23:16
False

2018-11-05 18:23:18    :     2018-11-05 18:23:17
False

2018-11-05 18:23:19    :     2018-11-05 18:23:18
False

2018-11-05 18:23:20    :     2018-11-05 18:23:19
False

2018-11-05 18:23:21    :     2018-11-05 18:23:20
False

2018-11-05 18:23:22    :     2018-11-05 18:23:21
False

2018-11-05 18:23:23    :     2018-11-05 18:23:22
False

2018-11-05 18:23:24    :     2018-11-05 18:23:23
False

2018-11-05 18:23:25    :     2018-11-05 18:23:24
False

2018-11-05 18:23:26    :     2018-11-05 18:23:25

### Awarie Lamp

In [35]:
# Zakładamy że Lampy co jakiś czas (minuta) pytają sąsiadów czy wszystko w porządku, jeśli sąsiad nie odpowiada przez kilka wiadomości podrząd, zakładamy że się zepsuł i raportujemy usterkę
# wybierz 2 lampy do awari i wylosuj kiedy się zepsują
# Lampy w promienu 50 metrów zauważają awarię po losowym okresie czasu (0-60s + 2 min) i zgłaszają ją do rozdzielni

# TODO : Usunąć zaświecane uszkodzonej lampy

import random
from  datetime import timedelta

lampsFail = pd.DataFrame(columns=['startTime','endTime','longitude','latitude'])
lampsAlert = pd.DataFrame(columns=['startTime','endTime','longitude','latitude'])



startDateTime = datetime.datetime.strptime(movement['time'].min(), "%Y-%m-%d %H:%M:%S")
endDateTime = datetime.datetime.strptime(movement['time'].max(), "%Y-%m-%d %H:%M:%S")
timeDelta = (endDateTime - startDateTime).total_seconds()

for i in range(2):
    victim = lamps.sample().iloc[0]
    timeOfDeath = startDateTime + timedelta(seconds=random.uniform(0, timeDelta))
    lampsFail.loc[len(lampsFail.index)] = [timeOfDeath, endDateTime, victim.X, victim.Y] 

    def lam(x):
        return geopy.distance.distance((victim.Y,victim.X),(x.Y, x.X)).m<50
    Neighbours = lamps.loc[lamps.apply(lam,axis=1)]
    #Neighbours = lamps[geopy.distance.distance((victim.Y,victim.X),(lamps['Y'], lamps['X'])).m<50]
    for index, n in Neighbours.iterrows():
        alertTime = timeOfDeath + timedelta(seconds=random.uniform(120, 180))
        if (victim.Y != n.Y and victim.X != n.X):
            lampsAlert.loc[len(lampsAlert.index)] = [alertTime, endDateTime, n.X, n.Y]


lampsFail.to_csv("lampFail.csv", index=False) 
lampsAlert.to_csv("lampAlert.csv", index=False) 

In [29]:
for index, row in Neighbours.iterrows():
    print(row.X)

19.9227298319221
19.9226656565028
19.9226009313466
19.9226706779405
19.9231274239757


In [6]:
#sourceDf.iloc[200:250]
movement[0:50]

,Unnamed: 0,time,longitude,latitude
1595,1595,2023-12-10 17:55:35,19.917040,50.064200
0,0,2023-12-10 17:55:39,19.909760,50.064610
1596,1596,2023-12-10 17:55:45,19.917140,50.064207
1,1,2023-12-10 17:55:49,19.909661,50.064626
1597,1597,2023-12-10 17:55:55,19.917240,50.064213
2,2,2023-12-10 17:55:59,19.909563,50.064642
1598,1598,2023-12-10 17:56:05,19.917339,50.064220
3,3,2023-12-10 17:56:09,19.909464,50.064659
1599,1599,2023-12-10 17:56:15,19.917438,50.064236
4,4,2023-12-10 17:56:19,19.909365,50.064675


In [8]:
type(startDateTime)

str

In [9]:
startDateTime

'2023-12-10 17:55:35'

In [33]:
df

,Unnamed: 0,time,course,haccuracy,latitude,longitude,speed,At,d,v
0,0,2018-11-16 07:44:12,0.0,36.448000,41.382380,2.176428,0.00,1.0,0.384526,0.384526
1,1,2018-11-16 07:44:13,0.0,22.512001,41.382381,2.176432,0.00,1.0,0.488646,0.488646
2,2,2018-11-16 07:44:14,0.0,15.008000,41.382385,2.176435,0.00,1.0,3.303497,3.303497
3,3,2018-11-16 07:44:15,50.1,11.792000,41.382414,2.176445,1.59,1.0,3.796754,3.796754
4,4,2018-11-16 07:44:16,41.4,13.936001,41.382444,2.176467,2.03,1.0,2.808925,2.808925
...,...,...,...,...,...,...,...,...,...,...
1072,1072,2018-11-16 08:02:04,94.1,34.304000,41.387430,2.187227,0.73,1.0,1.347109,1.347109
1073,1073,2018-11-16 08:02:05,59.6,31.088001,41.387441,2.187220,0.83,1.0,1.673570,1.673570
1074,1074,2018-11-16 08:02:06,1.3,40.736000,41.387455,2.187211,0.81,1.0,1.897636,1.897636
1075,1075,2018-11-16 08:02:07,332.8,45.024002,41.387468,2.187197,0.78,1.0,0.807752,0.807752
